In [3]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


In [2]:
# Load the CSV file
file_path = 'data/vgchartz-2024.csv'
data = pd.read_csv(file_path)

# Display basic info and first few rows
data.info()
data.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64016 entries, 0 to 64015
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   img           64016 non-null  object 
 1   title         64016 non-null  object 
 2   console       64016 non-null  object 
 3   genre         64016 non-null  object 
 4   publisher     64016 non-null  object 
 5   developer     63999 non-null  object 
 6   critic_score  6678 non-null   float64
 7   total_sales   18922 non-null  float64
 8   na_sales      12637 non-null  float64
 9   jp_sales      6726 non-null   float64
 10  pal_sales     12824 non-null  float64
 11  other_sales   15128 non-null  float64
 12  release_date  56965 non-null  object 
 13  last_update   17879 non-null  object 
dtypes: float64(6), object(8)
memory usage: 6.8+ MB


,img,title,console,genre,publisher,developer,critic_score,total_sales,na_sales,jp_sales,pal_sales,other_sales,release_date,last_update
0,/games/boxart/full_6510540AmericaFrontccc.jpg,Grand Theft Auto V,PS3,Action,Rockstar Games,Rockstar North,9.4,20.32,6.37,0.99,9.85,3.12,2013-09-17,NaN
1,/games/boxart/full_5563178AmericaFrontccc.jpg,Grand Theft Auto V,PS4,Action,Rockstar Games,Rockstar North,9.7,19.39,6.06,0.60,9.71,3.02,2014-11-18,2018-01-03
2,/games/boxart/827563ccc.jpg,Grand Theft Auto: Vice City,PS2,Action,Rockstar Games,Rockstar North,9.6,16.15,8.41,0.47,5.49,1.78,2002-10-28,NaN
3,/games/boxart/full_9218923AmericaFrontccc.jpg,Grand Theft Auto V,X360,Action,Rockstar Games,Rockstar North,NaN,15.86,9.06,0.06,5.33,1.42,2013-09-17,NaN
4,/games/boxart/full_4990510AmericaFrontccc.jpg,Call of Duty: Black Ops 3,PS4,Shooter,Activision,Treyarch,8.1,15.09,6.18,0.41,6.05,2.44,2015-11-06,2018-01-14


In [4]:
# Handle missing values
data = data.drop(columns=['img'])
data = data.dropna(subset=['total_sales', 'genre', 'publisher', 'critic_score'])

# Convert release_date to datetime
data['release_date'] = pd.to_datetime(data['release_date'], errors='coerce')

# Fill missing sales data with 0
sales_columns = ['na_sales', 'jp_sales', 'pal_sales', 'other_sales']
data[sales_columns] = data[sales_columns].fillna(0)

# Handle any remaining missing values
data = data.dropna()

data.info()
data.head()


<class 'pandas.core.frame.DataFrame'>
Index: 602 entries, 1 to 18917
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   title         602 non-null    object        
 1   console       602 non-null    object        
 2   genre         602 non-null    object        
 3   publisher     602 non-null    object        
 4   developer     602 non-null    object        
 5   critic_score  602 non-null    float64       
 6   total_sales   602 non-null    float64       
 7   na_sales      602 non-null    float64       
 8   jp_sales      602 non-null    float64       
 9   pal_sales     602 non-null    float64       
 10  other_sales   602 non-null    float64       
 11  release_date  602 non-null    datetime64[ns]
 12  last_update   602 non-null    object        
dtypes: datetime64[ns](1), float64(6), object(6)
memory usage: 65.8+ KB


,title,console,genre,publisher,developer,critic_score,total_sales,na_sales,jp_sales,pal_sales,other_sales,release_date,last_update
1,Grand Theft Auto V,PS4,Action,Rockstar Games,Rockstar North,9.7,19.39,6.06,0.60,9.71,3.02,2014-11-18,2018-01-03
4,Call of Duty: Black Ops 3,PS4,Shooter,Activision,Treyarch,8.1,15.09,6.18,0.41,6.05,2.44,2015-11-06,2018-01-14
7,Red Dead Redemption 2,PS4,Action-Adventure,Rockstar Games,Rockstar Games,9.8,13.94,5.26,0.21,6.21,2.26,2018-10-26,2018-11-02
8,Call of Duty: Black Ops II,X360,Shooter,Activision,Treyarch,8.4,13.86,8.27,0.07,4.32,1.20,2012-11-13,2018-04-07
9,Call of Duty: Black Ops II,PS3,Shooter,Activision,Treyarch,8.0,13.80,4.99,0.65,5.88,2.28,2012-11-13,2018-04-07


In [5]:
# Select features and target
features = ['genre', 'critic_score', 'publisher']
target = 'total_sales'

# Separate features and target
X = data[features]
y = data[target]

# Define preprocessing steps for numerical and categorical features
numerical_features = ['critic_score']
categorical_features = ['genre', 'publisher']

numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Apply the transformations
X_preprocessed = preprocessor.fit_transform(X)

# Convert the preprocessed data back to a DataFrame for inspection
X_preprocessed_df = pd.DataFrame(X_preprocessed.toarray())
X_preprocessed_df.head()


,0,1,2,3,4,5,6,7,8,9,...,95,96,97,98,99,100,101,102,103,104
0,1.619318,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.429050,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.693710,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.652225,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.354658,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# Save the preprocessed data for later use
X_preprocessed_df.to_csv('preprocessed_vgchartz_features.csv', index=False)
y.to_csv('preprocessed_vgchartz_target.csv', index=False)

X_preprocessed_df.info()
y.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 602 entries, 0 to 601
Columns: 105 entries, 0 to 104
dtypes: float64(105)
memory usage: 494.0 KB
<class 'pandas.core.series.Series'>
Index: 602 entries, 1 to 18917
Series name: total_sales
Non-Null Count  Dtype  
--------------  -----  
602 non-null    float64
dtypes: float64(1)
memory usage: 9.4 KB
